In [16]:
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC 
from selenium import webdriver 
from time import sleep


### Extraemos de la web del INE los datos del censo de 2022

In [17]:
import os
from time import sleep
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Calculamos la ruta absoluta de la carpeta "datos/ine" a partir de la carpeta donde se ejecuta el script
download_relative_path = os.path.join("datos", "ine")
download_dir = os.path.abspath(download_relative_path)

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": download_dir,  # Usamos la ruta absoluta
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "plugins.always_open_pdf_externally": True,
}

chrome_options.add_experimental_option("prefs", prefs)

url = "https://www.ine.es/jaxiT3/Datos.htm?t=68519"

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get(url)

# Aceptamos las cookies
sleep(2)
driver.find_element("css selector", "#aceptarCookie").click()

# Hacemos click en el botón de descarga
sleep(2)
driver.find_element("css selector", "#btnDescargaForm > i").click()
sleep(2)

# Seleccionamos CSV
iframe = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/div[6]/article/div/div/iframe"))
)
driver.switch_to.frame(iframe)
sleep(2)
driver.find_element("xpath", "/html/body/form[1]/ul/li[3]/label").click()

sleep(2)
driver.switch_to.default_content()

def wait_for_download_and_rename(download_folder, new_filename, wait_time=60):
    """
    Espera a que el archivo se descargue (es decir, que no tenga extensión .crdownload)
    y renombra el primer archivo con patrón 'Datos*.csv' al nombre 'new_filename'.
    """
    download_path = Path(download_folder)
    
    for _ in range(wait_time):
        # Obtenemos todos los archivos que se llamen '68519...' con extensión .csv
        # (ajusta el patron según el nombre real que devuelva el servidor)
        files = list(download_path.glob("68519*.csv"))
        
        # Verificamos que no exista ninguno con extensión .crdownload
        # (indica que la descarga no ha finalizado)
        crdownload_files = list(download_path.glob("*.crdownload"))
        
        if files and not crdownload_files:
            # Renombramos el primer archivo que encontremos
            old_file_path = files[0]
            
            # Construimos la nueva ruta
            new_file_path = download_path / new_filename
            
            # Si deseas conservar la extensión original o si el archivo es .xls, ajusta aquí
            # Por ejemplo, para CSV:
            if not new_file_path.suffix:
                new_file_path = new_file_path.with_suffix(".csv")
            
            # Realizamos el rename
            os.rename(old_file_path, new_file_path)
            print(f"Archivo renombrado a: {new_file_path}")
            return
        
        # Esperamos 1 segundo y seguimos buscando
        sleep(1)
    
    raise TimeoutError(
        "El archivo no se descargó o no pudo ser encontrado "
        "en el tiempo especificado."
    )

# ------------------------------------------------------------------------------

# Esperamos la descarga y renombramos a censo_2022.csv
wait_for_download_and_rename(download_dir, "censo_2022.csv")

# Cerrar el navegador
driver.quit()


Archivo renombrado a: c:\Users\Administrador\Desktop\RCM\TFM-analisis-educacion\datos\ine\censo_2022.csv
